In [1]:
from audio_diffusion_pytorch import AudioDiffusionModel
import torch
from torch import Tensor, nn, optim
from tqdm import tqdm
from IPython.display import Audio
from pathlib import Path
import torchaudio
import torchaudio.transforms as T
import pytorch_lightning as pl
from torch.utils.data import random_split, DataLoader, Dataset
import torch.nn.functional as F
import wandb
from pytorch_lightning.loggers import WandbLogger
from einops import rearrange

In [2]:
wandb_logger = WandbLogger(project="RemFX", save_dir="./")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mattricesound. Use `wandb login --relogin` to force relogin


In [3]:
SAMPLE_RATE = 22050
LENGTH = 2**17#round(5 * SAMPLE_RATE) 6 seconds

In [4]:
class GuitarDataset(Dataset):
    def __init__(self, root, length=LENGTH):
        self.files = list(Path().glob(f"{root}/**/*.wav"))
        self.resampler = T.Resample(48000, SAMPLE_RATE)
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        x, sr = torchaudio.load(self.files[idx])
#         x = x.view() # Duplicate channel
        resampled_x = self.resampler(x)
        if resampled_x.shape[1] < LENGTH:
            resampled_x = F.pad(resampled_x, (0, LENGTH - resampled_x.shape[1]))
        elif resampled_x.shape[1] > LENGTH:
            resampled_x = resampled_x[:, :LENGTH]
        return resampled_x

In [5]:
g = GuitarDataset(Path("Clean"))

train_split = 0.8
train_size = int(train_split * len(g))
val_size = len(g) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(g, [train_size, val_size])
train = DataLoader(train_dataset, batch_size=2)
val = DataLoader(val_dataset, batch_size=2)

In [6]:
dataiter = iter(train)
x = next(dataiter)
print(x.shape)
print(len(g), train_size, val_size)

torch.Size([2, 1, 131072])
690 552 138


In [7]:
# model = AudioDiffusionModel(in_channels=1)

In [8]:
class WrapperModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = AudioDiffusionModel(in_channels=1)
        
    def forward(self, x: torch.Tensor):
        return self.model(x)

    def sample(self, *args, **kwargs) -> Tensor:
        return self.model.sample(*args, **kwargs)
    
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx, mode="train")
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx, mode="val")

    def common_step(self, batch, batch_idx, mode: str = "train"):
        loss = self(batch)
        self.log(f"{mode}_loss", loss, on_step=True, on_epoch=True)
        return loss
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4, betas=(0.95, 0.999), eps=1e-6, weight_decay=1e-3)

    def on_validation_epoch_start(self):
        self.log_next = True
        
    def on_validation_batch_start(
        self, batch, batch_idx, dataloader_idx
    ):
        if self.log_next:
            self.log_sample(batch)
            self.log_next = False

    @torch.no_grad()
    def log_sample(self, batch, num_steps=10):
        # is_train = pl_module.training
        # if is_train:
            # pl_module.eval()
            
        # Get start diffusion noise
        noise = torch.randn(batch.shape, device=self.device)
        sampled = self.model.sample(
            noise=noise,
            num_steps=num_steps # Suggested range: 2-50
        ) 
        self.log_wandb_audio_batch(
            id="sample",
            samples=sampled,
            sampling_rate=SAMPLE_RATE,
            caption=f"Sampled in {num_steps} steps",
        )

        # if is_train:
            # pl_module.train()
    
    @torch.no_grad()
    def log_wandb_audio_batch(self, id: str, samples: Tensor, sampling_rate: int, caption: str = ""
    ):
        num_items = samples.shape[0]
        samples = rearrange(samples, "b c t -> b t c")
        for idx in range(num_items):
            wandb.log(
                {
                    f"sample_{idx}_{id}": wandb.Audio(
                        samples[idx].cpu().numpy(),
                        caption=caption,
                        sample_rate=sampling_rate,
                    )

                }
            )


In [9]:
model = WrapperModel()

In [10]:
trainer = pl.Trainer(limit_train_batches=100, max_epochs=1000, accelerator='gpu', devices=[1], logger=wandb_logger)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
trainer.fit(model=model, train_dataloaders=train, val_dataloaders=val)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name  | Type                | Params
----------------------------------------------
0 | model | AudioDiffusionModel | 106 M 
----------------------------------------------
106 M     Trainable params
0         Non-trainable params
106 M     Total params
424.786   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/jovyan/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/jovyan/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1000` reached.


In [59]:
noise = torch.randn(2, 1, LENGTH)
sampled = model.sample(
    noise=noise,
    num_steps=10 # Suggested range: 2-50
) # [2, 1, 2 ** 18]

In [60]:
Audio(sampled[0], rate=SAMPLE_RATE)

In [54]:
test = "hi"

In [56]:
print(test)

hi


In [62]:
Audio(x[1], rate=SAMPLE_RATE)